# AKSSF Anadromous Waters Catalog Data Join
Use spatial join to add AWC_CODE, Species and LSTAGE information to AKSSF catchments.  Add new field to concatenate species+lstage and field ot designate salmon spawning habitat presence or absence.

In [1]:
import os, arcpy, sys,datetime, traceback
import time

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = os.path.dirname(os.getcwd())
print(f'CSV table output directory set to {outdir}')

imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\A

## Functions

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

## Section 1
### Set input datasets and output locations
User to input paths for necessary input data and output locations



In [3]:
# Get user inputs
# Used to format user inputs
inputDict = {"'":"",'"':""}# Specify path to AKSSF parent directory
while True:
    try:
        userinput = replace_all((input('Input path to AWC HUC12 catchments outlets fc.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_AWC_HUC12_outlet_cats_merge') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_AWC_HUC12_outlet_cats_merge"),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            catsint = userinput
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AKSSF AWC HUC12 outlet catchments set to {catsint}\n {"-"*100}')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events_source = userinput2
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC events feature class set to {awc_events_source}\n {"-"*100}')

# Specify path to AWC Lakes events fc
while True:
    try:
        userinput4 = replace_all((input('Input path to awc lakes events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents"), inputDict)
        if not arcpy.Exists(userinput4):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_lakes = userinput4
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC lakes events feature class set to {awc_lakes}\n {"-"*100}')

# Specify path to output gdb
while True:
    try:
        userinput3 = replace_all((input('Input path to output gdb.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb"), inputDict)
        if not arcpy.Exists(userinput3):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            outputgdb = userinput3
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'Output GDB set to {outputgdb}\n {"-"*100}')

# Specify path to sites spatial join fc
while True:
    try:
        siteinput = replace_all((input('Input path to temp sites fc.\nLeave blank and hit enter to use the default location.\nDefault = U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_points') or "U:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\AKSSF_awcHuc12_outlet_cats_points"),inputDict)
        if not arcpy.Exists(siteinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            sites = siteinput
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AKSSF AWC HUC12 catchment outlet centroids {sites}\n {"-"*100}')



AKSSF AWC HUC12 outlet catchments set to D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_AWC_HUC12_outlet_cats_merge
 ----------------------------------------------------------------------------------------------------
AWC events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\awcEventArcs
 ----------------------------------------------------------------------------------------------------
AWC lakes events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\AWCLakeEvents
 ----------------------------------------------------------------------------------------------------
Output GDB set to D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb
 ----------------------------------------------------------------------------------------------------
AKSSF AWC HUC12 catchment outlet centroids D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_outlet_cats_points
 --------------------------------------------------

## Join awc data to catchments via spatial join
* During spatial join add new field 'AWC_OID' and use match_type = join on awc=Events ObjectID so sp/ls can be queried from source polyline, not from AWC_CODE
    * Must copy to memory and second oid field to merge because source OID field will not merge with join option - just copies OID from output fc to this joined field?
* Add field 'sp_ls' to record species and l_stage comma seperated concatenations and field 'sa_sp_hab' to record presence absence of salmon spawning habitat

In [4]:
# Make copy of awcEvents in memory and add second OID field because original field will not honor the merge rule 'join' in arcpy spatial join (works in Pro?)
awc_events = arcpy.FeatureClassToFeatureClass_conversion(awc_events_source,r'memory','awcEventsCopy')
arcpy.AddField_management(awc_events,'OID2','LONG')
with arcpy.da.UpdateCursor(awc_events,['OBJECTID','OID2']) as cur:
    for row in cur:
        row[1] = row[0]
        cur.updateRow(row)
    del(row)
del(cur)
print(f'AWC Copy complete\n{"*"*100}')

AWC Copy complete
****************************************************************************************************


In [5]:
import time, datetime
arcpy.env.overwriteOutput = True
# Parameters
joinopt = "JOIN_ONE_TO_ONE"
jointype = "KEEP_ALL"
matchopt = "INTERSECT"
s_rad = "1 Centimeters"
processStart = time.time()

print(f'Joining species and lifestage data from {awc_events} to {catsint}')
awcjoin = arcpy.analysis.SpatialJoin(catsint,
                                     awc_events,
                                     os.path.join(outputgdb,'AKSSF_all_cats_int_awc_spJoin'),
                                     joinopt,
                                     jointype,
                                     r'cat_ID_con "cat_ID_con" true true false 255 Text 0 0,First,#,catsint,cat_ID_con,0,255;'
                                     r'HUC12 "HUC12" true true false 255 Text 0 0,First,#,catsint,HUC12,0,255;'
                                     r'MERGE_SRC "MERGE_SRC" true true false 255 Text 0 0,First,#,catsint,MERGE_SRC,0,255;'
                                     # Add AWC_OID field so Sp/Ls records can be queried from polyine rather than AWC_CODE
                                     r'AWC_OID "AWC_OID" true true false 500 Text 0 0,Join,",",awcEventArcs,OID2,-1,-1;'
                                     r'AWC_CODE "AWC_CODE" true false false 500 Text 0 0,Join,",",awc_events,AWC_CODE,0,70;'
                                     r'SPECIES "SPECIES" true false false 500 Text 0 0,Join,",",awc_events,SPECIES,0,2;'
                                     r'LSTAGE "LSTAGE" true false false 500 Text 0 0,Join,",",awc_events,LSTAGE,0,1',
                                     matchopt,
                                     s_rad,
                                     ''
                                     )

# ADD additional fields
arcpy.AddField_management(awcjoin,'sp_ls','TEXT')
arcpy.AddField_management(awcjoin,'sa_sp_hab','SHORT')
arcpy.AddField_management(awcjoin,'sa_sp_count','SHORT')
arcpy.AddField_management(awcjoin,'mult_awc','SHORT')
arcpy.AddField_management(awcjoin,'AWC_CODE_qc','TEXT',"","",500)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')


Joining species and lifestage data from memory\awcEventsCopy to D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_AWC_HUC12_outlet_cats_merge
Process complete
Process completed at 2022-04-15 09:27 (Elapsed time: 0:02:50)
****************************************************************************************************



## ~~Make dictionary from awc events~~
## Cannot use a dictionary because the awc records from spatial join do not match records in value Dictionary for all systems.



In [23]:
awcDict = {}
with arcpy.da.SearchCursor(awc_events, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcDict,row[0],sp_ls_val)

awcOidDict = {}
with arcpy.da.SearchCursor(awc_events, ['OBJECTID',"SPECIES","LSTAGE","AWC_CODE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcOidDict,row[0],[sp_ls_val,row[3]])

awcLakesDict = {}
with arcpy.da.SearchCursor(awc_lakes, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcLakesDict,row[0],sp_ls_val)
print(f'AWC Dictionaries Created\n{("*"*100)}')


AWC Dictionaries Created
****************************************************************************************************


## Use update cursor to populate new field
sa_sp_hab: 1 = spawning, 2 = salmon species present but not spawning, null = no awc records
sa_sp_count: # of salmon species listed as spawning, null for no data
mult_awc: 1 for multiple awc codes, 0 for one awc code and null for no data

In [8]:
awcjoin = r"D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_all_cats_int_awc_spJoin"
dropfields = ['Join_Count', 'NHDPlusID','catID','Shape','TARGET_FID','SOURCEFC','region']
keepfields = [f.name for f in arcpy.ListFields(awcjoin) if f.name not in dropfields]
sa_sp_codes = ['CO_s','S_s','K_s', 'CH_s','P_s']
sa_no_sp_codes = ['CO_p','CO_r','S_p','S_r','K_p','K_r','CH_p','CH_r','P_r','P_p']
print(keepfields)
multirecslist = []
multirecsDict = {}

norecslist = []
sp_lsList = []
awcCodeDict = {}
catAwcDict = {}
curfields = ['cat_ID_con','AWC_CODE','Species', 'LSTAGE', 'sp_ls','sa_sp_hab','mult_awc','AWC_CODE_qc','AWC_OID','sa_sp_count']
c2 = 0
processStart = time.time()
with arcpy.da.UpdateCursor(awcjoin, curfields) as cur:
    for row in cur:
        awcvals = []
        print(f'{(100*"*")}')
        if row[1] is not None:
            for val in row[1].split(','):
                val2 = "'"+str(val)+"'"
                awcvals.append(val)
            awcvals = list(set(awcvals))
            if len(str(awcvals).split(',')) > 1:
                print(f'Multiple awc records({len(str(awcvals).split(","))}) for this catchment ')
                append_value(multirecsDict,row[0],awcvals)
                row[6] = 1
                row[7] = str(awcvals).strip("[]").replace("'","")
                multirecslist .append(row[0])
            else:
                row[7]=awcvals[0]
                row[6] = 0
            spvals = row[2].split(",")
            lsvals = row[3].split(",")
            snum = len(row[2].split(","))
            cat_spList =[]
            for i in range(snum):
                sp_ls = spvals[i] + '_' + lsvals[i]
                append_value(awcCodeDict,row[1],sp_ls)
                append_value(catAwcDict,row[0],sp_ls)
                sp_lsList.append(sp_ls)
                cat_spList.append(sp_ls)
            print(f'{row[0]} has {snum} species_ls records {cat_spList}')
            print(f'list without duplicate records {list(set(cat_spList))}')
            row[4] = str(list(set(cat_spList))).strip("[]").replace("'","")
            if len(set(cat_spList).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {set(cat_spList).intersection(sa_sp_codes)} salmon spawning records')
                saspcount = len(set(cat_spList).intersection(sa_sp_codes))
                row[5] = 1
                row[9] = saspcount
            elif len(set(cat_spList).intersection(sa_no_sp_codes)) > 0  and len(set(cat_spList).intersection(sa_sp_codes)) == 0:
                print(f'{row[0]} has {len(set(cat_spList).intersection(sa_no_sp_codes))} salmon species records')
                row[5] = 2
                row[9] = 0
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[5] = 0
                row[9] = 0
        else:
            print(f'{row[0]} did not intersect the AWC ')
            row[1]=None
            row[2]=None
            row[3]=None
            row[4]=None
            row[5]=None
            row[6]=None
            row[7]=None
            row[8]=None
            row[9]=None
            norecslist.append(row[0])
            c2+=1
        cur.updateRow(row)
    del(row)
del(cur)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')

print(f'{c2} catchments did not have AWC species records')
print(norecslist)

['OBJECTID', 'cat_ID_con', 'HUC12', 'MERGE_SRC', 'AWC_OID', 'AWC_CODE', 'SPECIES', 'LSTAGE', 'Shape_Length', 'Shape_Area', 'sp_ls', 'sa_sp_hab', 'sa_sp_count', 'mult_awc', 'AWC_CODE_qc']
****************************************************************************************************
Cook_Inlet_75004200000901 has 1 species_ls records ['K_r']
list without duplicate records ['K_r']
Cook_Inlet_75004200000901 has 1 salmon species records
****************************************************************************************************
Cook_Inlet_75004200001724 has 3 species_ls records ['K_p', 'CO_r', 'S_s']
list without duplicate records ['S_s', 'K_p', 'CO_r']
Cook_Inlet_75004200001724 has {'S_s'} salmon spawning records
****************************************************************************************************
Cook_Inlet_75004200000437 has 2 species_ls records ['P_p', 'CH_p']
list without duplicate records ['P_p', 'CH_p']
Cook_Inlet_75004200000437 has 2 salmon species record

## Qc and update results
Update using qc dictionaries from initial review, 2nd and final.
Rebuild watersheds before running script below and review again/create new watersheds as necessary.
Some outlet catchments appear to wrong when compared to the flow accumulation grid but when compared to values stored in the vaa_merge (from which they are identified) you can see that the catchment identified is the one with the maximum TotDASqKM value...I think in most cases it makes sense to move the outlet catchment to the catchment that matches the fac grid/imagery unless the new watershed that will be calculated as a result of the change will not be significantly different from the watershed that was already created.

Example of the vaa vs flow acc grid discrepancy (calculated values from fac count much different than vaa totdasqkm value):
 * TotDASqKm of Cook_Inlet_75005300002575 = 2028.669626 but fac value @ 1,142,521 (1142521 x 25sqm x 1e-6 = 28.56 sqkm) # outlet catchment identified from vaa table values

 * TotDASqKm of Cook_Inlet_75005300009967 = 2000.180576 but fac value @ 21,512,896 (21512896 x 25sqm x 1e-6 = 537.82 sqkm) # catchment that should be outlet based on visual examination and flow acc grid values

 * Check of Cook_Inlet_75005300016711 = 41.32sqkm , fac @ 1,405,319 (1405319 x 25sqm x 1e-6 = 35.13)
 * Check of Cook_Inlet_75004400008887 = 11.45sqkm, fac @ 458,122 (458122 x 25sqm x 1e-6 = 11.45)

## Check Results
Check Multiple codes catchment - Cook_Inlet_75005300002504 "Straight Creek" had 2 AWC-Codes associated with the catchment during initial Spatial Join and should have the following records after qc update:
from manual Count : K_r, CH_p, S_r, CO_p, DV_r, P_p, DV_s
from QC script : K_r, CH_p, S_r, CO_p, DV_r, P_p, DV_s

Check missed during spatial join - 'Copper_River_75003900054448' AWC record begins above the outlet catchment:
Manual Count : P_p, CO_r, CT_s, CT_r, CO_s
QC Script result: P_p, CO_r, CT_s, CT_r, CO_s

Check new Watershed - 'Prince_William_Sound_77992':['Prince_William_Sound_75842'] where 75842 will be the new watershed/outlet catchment for the HUC that initially had 77992 as the outlet.
Manual Count : P_s, Co_s, CO_r
QC Script result: P_s, CO_s, CO_r

Check new Watershed that did not intersect AWC - 'Copper_River_75003900054944':['Copper_River_75003900054448']
Manual Count : P_p, CO_r, CT_s, CT_r, CO_s
QC Script result: P_p, CO_r, CT_s, CT_r, CO_s


### Compare dictionaries and use to set definition queries for review.

In [61]:
# alreadyQCList = []
# for k, v in awcHuc12QcDict.items():
#     print (f'"{k}",')
#     alreadyQCList.append(k)
# print(len(norecslist))
# print(len(multirecslist))
# qcList = norecslist + multirecslist
# print (len(qcList))
# print (f'qcList: {len(qcList)}') # List of all watersheds/catchments that need to be reviewed for either wrong catchment or multi_awc records.
# upQcList = [x for x in qcList if x not in alreadyQCList]
# print(len(upQcList))
# print(upQcList)
#
# # Drops from initial review.  Add these to review dictionary below
# dropList = ['Cook_Inlet_75004300001512', 'Cook_Inlet_75004300000779', 'Cook_Inlet_75004400008539', 'Cook_Inlet_75004400009109', 'Cook_Inlet_75004400002718', 'Cook_Inlet_75000100002639', 'Cook_Inlet_75000100004624', 'Cook_Inlet_75000700037019', 'Cook_Inlet_75000500015568', 'Cook_Inlet_75000500006160', 'Cook_Inlet_75000500009986', 'Cook_Inlet_75000300007777', 'Cook_Inlet_75000600014946', 'Cook_Inlet_75000600020425', 'Cook_Inlet_75000200015292', 'Cook_Inlet_75005300000657', 'Cook_Inlet_75005400045669', 'Cook_Inlet_75005400028381', 'Cook_Inlet_75005400023457', 'Cook_Inlet_75005400047280', 'Copper_River_75019800002596', 'Copper_River_75019800019137', 'Copper_River_75019800020572', 'Copper_River_75019800000761', 'Copper_River_75019600178680', 'Copper_River_75019600201122', 'Copper_River_75019600145942', 'Copper_River_75019600119614', 'Copper_River_75019600130736', 'Copper_River_75019600198047', 'Copper_River_75019700000617', 'Copper_River_75019700000923', 'Copper_River_75019700003395', 'Copper_River_75003900073776', 'Copper_River_75003900066004', 'Copper_River_75003900054248', 'Copper_River_75003900029086', 'Prince_William_Sound_33616', 'Prince_William_Sound_91751']
#
# upQcList2 = [x for x in upQcList if x not in dropList]
# #print(upQcList2)
# #format for key and print to add
# for item in dropList:
#     print(f"'{item}':None,")


"Cook_Inlet_75004200004105",
"Cook_Inlet_75004400002041",
"Prince_William_Sound_46513",
"Cook_Inlet_75000200007660",
"Cook_Inlet_75004400011322",
"Cook_Inlet_75004300006013",
"Cook_Inlet_75004400010320",
"Copper_River_75019800010313",
"Cook_Inlet_75004400010754",
"Copper_River_75019700004190",
"Cook_Inlet_75004400010402",
"Cook_Inlet_75004300001162",
"Cook_Inlet_75000100002463",
"Prince_William_Sound_43933",
171
163
334
qcList: 334
320
['Cook_Inlet_75004200001726', 'Cook_Inlet_75004200001298', 'Cook_Inlet_75004200003619', 'Cook_Inlet_75004200016346', 'Cook_Inlet_75004200006640', 'Cook_Inlet_75004200012680', 'Cook_Inlet_75004200005429', 'Cook_Inlet_75004200012771', 'Cook_Inlet_75004200009218', 'Cook_Inlet_75004200013577', 'Cook_Inlet_75004200000478', 'Cook_Inlet_75004200006975', 'Cook_Inlet_75004200001167', 'Cook_Inlet_75004300006970', 'Cook_Inlet_75004300002057', 'Cook_Inlet_75004300007319', 'Cook_Inlet_75004300007978', 'Cook_Inlet_75004300002143', 'Cook_Inlet_75004300001135', 'Cook_In

In [59]:
# Dictionary of all catchments to be reviewed either for Multiple code associations, new catchments/watersheds or just missed during spatial join
awcHuc12QcDict = {'Cook_Inlet_75004400002041': '244-30-10010-2177-3012',
                  'Cook_Inlet_75000200007660': '247-41-10100',
                  'Cook_Inlet_75004400011322': '247-60-10170-2029-3033',
                  'Cook_Inlet_75004300006013': '244-30-10050-2059',
                  'Cook_Inlet_75004400010320': '244-30-10010-2030',
                  'Copper_River_75019800010313': '212-20-10080-2655',
                  'Cook_Inlet_75004400010754': '244-30-10010-2158',
                  'Copper_River_75019700004190': '212-20-10080-2300-3421',
                  'Cook_Inlet_75004400010402': '244-30-10010-2025-3041',
                  'Cook_Inlet_75004300001162': '244-10-10010-2011',
                  'Cook_Inlet_75000100002463': '247-50-10060',
                  'Prince_William_Sound_43933': '221-20-10360',
                  'Cook_Inlet_75004200001493': '233-20-10080',
                  'Cook_Inlet_75004200000064': '231-30-10080-2010',
                  'Cook_Inlet_75004300004365': '244-20-10100-2019',
                  'Cook_Inlet_75004300003713': '244-20-10100',
                  'Cook_Inlet_75004300002755': '244-20-10100-2045',
                  'Cook_Inlet_75004300002023': '241-14-10645',
                  'Cook_Inlet_75004300002027': '241-14-10645-2060',
                  'Cook_Inlet_75004300001228': '244-10-10010-2011-3031',
                  'Cook_Inlet_75004400011230': '244-30-10010-2250-3030',
                  'Cook_Inlet_75004400002953': '247-90-10020-2058',
                  'Cook_Inlet_75004400007512': '244-30-10010-2063',
                  'Cook_Inlet_75004400009198': '247-60-10110-2120',
                  'Cook_Inlet_75004400011676': '247-60-10220',
                  'Cook_Inlet_75004400004680': '244-30-10010-2076',
                  'Cook_Inlet_75004400001464': '247-60-10170-2029-3041',
                  'Cook_Inlet_75004400001136': '247-60-10250',
                  'Cook_Inlet_75004400010328': '244-30-10010',
                  'Cook_Inlet_75004400004743': '247-60-10100',
                  'Cook_Inlet_75004400010352': '247-60-10150-2025',
                  'Cook_Inlet_75004400010337': '244-30-10010-2063-3041',
                  'Cook_Inlet_75004400010221': '247-60-10230',
                  'Cook_Inlet_75004400009412': '244-30-10010-2177',
                  'Cook_Inlet_75004400005344': '244-30-10010-2225-3031',
                  'Cook_Inlet_75004400008192': '247-60-10170-2029',
                  'Cook_Inlet_75004400002258': '247-60-10280',
                  'Cook_Inlet_75004400008859': '247-90-10020',
                  'Cook_Inlet_75004400000153': '247-90-10020',
                  'Cook_Inlet_75004400004707': '247-60-10110-2080',
                  'Cook_Inlet_75004400007493': '244-30-10010-2050',
                  'Cook_Inlet_75004400002957': '244-30-10010-2063-3036',
                  'Cook_Inlet_75000100001880': '247-60-10340-2018',
                  'Cook_Inlet_75000100002688': '247-50-10150',
                  'Cook_Inlet_75000100004381': '247-60-10320',
                  'Cook_Inlet_75000100002741': '247-50-10110-2070',
                  'Cook_Inlet_75000100000953': '247-50-10175-2025',
                  'Cook_Inlet_75000400034535': '247-50-10200-2160',
                  'Cook_Inlet_75000400001221': '247-50-10220',
                  'Cook_Inlet_75000700033509': '247-41-10200-2391',
                  'Cook_Inlet_75000700049477': '247-41-10200-2551',
                  'Cook_Inlet_75000700036340': '247-41-10200-2596',
                  'Cook_Inlet_75000700044263': '247-41-10200',
                  'Cook_Inlet_75000700049829': '247-41-10200-2696-3020',
                  'Cook_Inlet_75000700036399': '247-41-10200-2585-3223',
                  'Cook_Inlet_75000500008473': '247-41-10200-2381-3161-4016',
                  'Cook_Inlet_75000500009358': '247-41-10200-2381-3235',
                  'Cook_Inlet_75000500005048': '247-41-10200-2381-3231',
                  'Cook_Inlet_75000500013555': '247-41-10200-2381-3240',
                  'Cook_Inlet_75000500008026': '247-41-10200-2381',
                  'Cook_Inlet_75000300001605': '247-41-10200-2370-3301-4044',
                  'Cook_Inlet_75000300007234': '247-41-10200-2370-3015',
                  'Cook_Inlet_75000300018625': '247-41-10200-2370',
                  'Cook_Inlet_75000300025330': '247-41-10200-2370',
                  'Cook_Inlet_75000600003641': '247-41-10200-2053-3250',
                  'Cook_Inlet_75000600035022': '247-41-10200-2053-3213',
                  'Cook_Inlet_75000600024655': '247-41-10200-2053-3205-4053-5066',
                  'Cook_Inlet_75000600034967': '247-41-10200-2053-3205-4053-5066',
                  'Cook_Inlet_75000600040180': '247-41-10200-2053-3205-4053-5072',
                  'Cook_Inlet_75000600025547': '247-41-10200-2053',
                  'Cook_Inlet_75000600004193': '247-41-10200-2053-3205-4053-5046-6011',
                  'Cook_Inlet_75000600009435': '247-41-10200-2053-3205-4053-5046',
                  'Cook_Inlet_75000600014556': '247-41-10200-2053-3220-4030-5040',
                  'Cook_Inlet_75000600025489': '247-41-10200-2053-3229-4050',
                  'Cook_Inlet_75000600034555': '247-41-10200-2053-3238',
                  'Cook_Inlet_75000600023013': '247-41-10200-2053-3229',
                  'Cook_Inlet_75000600034913': '247-41-10200-2053-3205-4053-5028',
                  'Cook_Inlet_75000600015366': '247-41-10200-2053-3205',
                  'Cook_Inlet_75000600022522': '247-41-10200-2053-3205-4112',
                  'Cook_Inlet_75000600040654': '247-41-10200-2053-3150-4018',
                  'Cook_Inlet_75000600014918': '247-41-10200-2053-3205-4009',
                  'Cook_Inlet_75000600014428': '247-41-10200-2053-3205-4064',
                  'Cook_Inlet_75000600036866': '247-41-10200-2053-3205-4112-5037',
                  'Cook_Inlet_75000600012683': '247-41-10200-2053-3205',
                  'Cook_Inlet_75000600005715': '247-41-10200-2053-3205-4112-5060',
                  'Cook_Inlet_75000600020443': '247-41-10200-2053-3170-4057',
                  'Cook_Inlet_75000600020381': '247-41-10200-2053-3150-4060-5033',
                  'Cook_Inlet_75000200008600': '247-41-10200-2200-3050',
                  'Cook_Inlet_75000200018499': '247-41-10200-2291-3011',
                  'Cook_Inlet_75000200015565': '247-41-10200-2081-3065-4021',
                  'Cook_Inlet_75000200005720': '247-41-10200-2081-3050-4040',
                  'Cook_Inlet_75000200001253': '247-41-10200-2120',
                  'Cook_Inlet_75000200001139': '247-41-10200-2250-3061-4009',
                  'Cook_Inlet_75000200008048': '247-41-10200-2081-3100-4155',
                  'Cook_Inlet_75000200015589': '247-41-10200-2020',
                  'Cook_Inlet_75000200015485': '247-41-10200-2015-3025',
                  'Cook_Inlet_75000200015301': '247-41-10200-2015-3035',
                  'Cook_Inlet_75000200005066': '247-41-10200-2015-3040',
                  'Cook_Inlet_75000200008267': '247-41-10200',
                  'Cook_Inlet_75000200013093': '247-41-10200-2053-3020-4015',
                  'Cook_Inlet_75005300026455': '247-20-10010-2040',
                  'Cook_Inlet_75005300002504': '247-10-10080-2010-3040',
                  'Cook_Inlet_75005300030509': '247-30-10090-2105-3015',
                  'Cook_Inlet_75005300019114': '247-30-10080',
                  'Cook_Inlet_75005300026951': '247-10-10080',
                  'Cook_Inlet_75005300026568': '247-20-10010-2020',
                  'Cook_Inlet_75005300030350': '247-20-10010-2052',
                  'Cook_Inlet_75005300021072': '245-50-10010-2047',
                  'Cook_Inlet_75005300003835': '247-20-10010',
                  'Cook_Inlet_75005300022240': '247-10-10200',
                  'Cook_Inlet_75005300024421': '245-50-10050',
                  'Cook_Inlet_75005400015980': '245-10-10030-2007',
                  'Cook_Inlet_75005400041329': '245-40-10020',
                  'Copper_River_75019800005498': '212-20-10080-2481-3181',
                  'Copper_River_75019800011243': '212-20-10080-2571-3062-4042',
                  'Copper_River_75019800010136': '212-20-10080-2605-3019',
                  'Copper_River_75019800006994': '212-20-10080-2571-3062',
                  'Copper_River_75019800009388': '212-20-10080-2605',
                  'Copper_River_75019800020803': '212-20-10080-2576',
                  'Copper_River_75019800000492': '212-20-10080-2481-3071',
                  'Copper_River_75019800003840': '212-20-10080-2481',
                  'Copper_River_75019800011423': '212-20-10080-2755',
                  'Copper_River_75019800011406': '212-20-10080-2500',
                  'Copper_River_75019800011978': '212-20-10080-2481',
                  'Copper_River_75019600164515': '212-20-10080-2461-3171',
                  'Copper_River_75019600143415': '212-20-10080-2461-3091',
                  'Copper_River_75019600187749': '212-20-10080-2461-3091-4122',
                  'Copper_River_75019600201378': '212-20-10080-2461-3091-4042',
                  'Copper_River_75019600117650': '212-20-10080-2461-3091',
                  'Copper_River_75019600187874': '212-20-10080-2461-3091',
                  'Copper_River_75019600165863': '212-20-10080-2461-3091-4021',
                  'Copper_River_75019600118256': '212-20-10080-2461',
                  'Copper_River_75019600113018': '212-20-10080-2461-3023',
                  'Copper_River_75019600198311': '212-20-10080-2431-3051',
                  'Copper_River_75019600117903': '212-20-10080',
                  'Copper_River_75019600128647': '212-20-10080',
                  'Copper_River_75019600195594': '212-20-10080-2331-3060',
                  'Copper_River_75019600132577': '212-20-10080-2331-3068',
                  'Copper_River_75019600119855': '212-20-10080-2331-3071',
                  'Copper_River_75019600133541': '212-20-10080-2331',
                  'Copper_River_75019600112990': '212-20-10080-2331-3081-4050',
                  'Copper_River_75019700004212': '212-20-10080-2300-3540-4100',
                  'Copper_River_75019700001727': '212-20-10080-2300-3310-4085',
                  'Copper_River_75019700004866': '212-20-10080-2300-3540',
                  'Copper_River_75019700017989': '212-20-10080-2300',
                  'Copper_River_75019700005942': '212-20-10080-2300-3421-4021',
                  'Copper_River_75019700010631': '212-20-10080-2300-3511',
                  'Copper_River_75019700003637': '212-20-10080-2300-3371-4041',
                  'Copper_River_75003900057793': '212-20-10080',
                  'Copper_River_75003900034254': '212-20-10080-2159',
                  'Copper_River_75003900051444': '212-20-10040',
                  'Copper_River_75003900023132': '212-10-10010-2061',
                  'Prince_William_Sound_18547': '227-10-17060-2012',
                  'Prince_William_Sound_18737': '227-30-17790',
                  'Prince_William_Sound_30886': '227-20-17386',
                  'Prince_William_Sound_41995': '227-20-17590-2003',
                  'Prince_William_Sound_43195': '228-60-18120',
                  'Prince_William_Sound_45835': '224-40-14870',
                  'Prince_William_Sound_74542': '222-50-12570',
                  'Prince_William_Sound_79792': '223-30-13220',
                  'Prince_William_Sound_91741': '221-60-11370',
                  'Prince_William_Sound_93261': '221-60-11470',
                  'Copper_River_75019700004288': None,
                  'Cook_Inlet_75004400008887': None,
                  'Cook_Inlet_75000100002639': None,
                  'Cook_Inlet_75004300001512': None,
                  'Cook_Inlet_75004300000779': None,
                  'Cook_Inlet_75004400008539': None,
                  'Cook_Inlet_75004400009109': None,
                  'Cook_Inlet_75004400002718': None,
                  'Cook_Inlet_75000100004624': None,
                  'Cook_Inlet_75000700037019': None,
                  'Cook_Inlet_75000500015568': None,
                  'Cook_Inlet_75000500006160': None,
                  'Cook_Inlet_75000500009986': None,
                  'Cook_Inlet_75000300007777': None,
                  'Cook_Inlet_75000600014946': None,
                  'Cook_Inlet_75000600020425': None,
                  'Cook_Inlet_75000200015292': None,
                  'Cook_Inlet_75005300000657': None,
                  'Cook_Inlet_75005400045669': None,
                  'Cook_Inlet_75005400028381': None,
                  'Cook_Inlet_75005400023457': None,
                  'Cook_Inlet_75005400047280': None,
                  'Copper_River_75019800002596': None,
                  'Copper_River_75019800019137': None,
                  'Copper_River_75019800020572': None,
                  'Copper_River_75019800000761': None,
                  'Copper_River_75019600178680': None,
                  'Copper_River_75019600201122': None,
                  'Copper_River_75019600145942': None,
                  'Copper_River_75019600119614': None,
                  'Copper_River_75019600130736': None,
                  'Copper_River_75019600198047': None,
                  'Copper_River_75019700000617': None,
                  'Copper_River_75019700000923': None,
                  'Copper_River_75019700003395': None,
                  'Copper_River_75003900073776': None,
                  'Copper_River_75003900066004': None,
                  'Copper_River_75003900054248': None,
                  'Copper_River_75003900029086': None,
                  'Prince_William_Sound_33616': None,
                  'Prince_William_Sound_91751': None,
                  'Cook_Inlet_75000200005366': None,
                  'Copper_River_75003900043422': None,
                  'Copper_River_75003900033530': None,
                  'Cook_Inlet_75004300002143': None,
                  'Cook_Inlet_75000100000043': None,
                  'Cook_Inlet_75000100002150': None,
                  'Copper_River_75003900029096': None,
                  'Prince_William_Sound_46513': None,
                  'Cook_Inlet_75004200003619': '232-22-10130',
                  'Cook_Inlet_75004200016346': '231-20-10337',
                  'Cook_Inlet_75004200012680': '233-30-10070',
                  'Cook_Inlet_75004200005429': '231-30-10080-2119',
                  'Cook_Inlet_75004200012771': '242-31-10120',
                  'Cook_Inlet_75004200006975': '226-50-16286',
                  'Cook_Inlet_75004300006970': '241-14-10645',
                  'Cook_Inlet_75004300002057': '241-14-10645-2131',
                  'Cook_Inlet_75004300007978': '241-14-10600',
                  'Cook_Inlet_75004300001135': '241-30-10500',
                  'Cook_Inlet_75004300006302': '241-20-10550',
                  'Cook_Inlet_75004300002183': '241-40-10300',
                  'Cook_Inlet_75004300005952': '241-14-10660',
                  'Cook_Inlet_75004400007224': '247-60-10210',
                  'Cook_Inlet_75004400000334': '244-30-10010',
                  'Cook_Inlet_75004400000370': '244-30-10010-2140',
                  'Cook_Inlet_75004400003230': '244-30-10010-2076-3095',
                  'Cook_Inlet_75004400001670': '244-30-10010-2096',
                  'Cook_Inlet_75000100004048': '247-50-10175',
                  'Cook_Inlet_75000100002482': '247-41-10080',
                  'Cook_Inlet_75000100004383': '247-50-10330',
                  'Cook_Inlet_75000400014408': '247-50-10200',
                  'Cook_Inlet_75000400000017': '247-50-10220',
                  'Cook_Inlet_75000400000163': '247-50-10220-2095',
                  'Cook_Inlet_75000700031304': '247-41-10200-2713',
                  'Cook_Inlet_75000700046720': '247-41-10200',
                  'Cook_Inlet_75000500006217': '247-41-10200-2381-3161',
                  'Cook_Inlet_75000500002477': '247-41-10200-2381-3161',
                  'Cook_Inlet_75000600029695': '247-41-10200-2053-3170-4093',
                  'Cook_Inlet_75000600027457': '247-41-10200-2053-3229',
                  'Cook_Inlet_75000600002850': '247-41-10200-2053-3205-4075-5255',
                  'Cook_Inlet_75000600020073': '247-41-10200-2053-3205',
                  'Cook_Inlet_75000600027807': '247-41-10200-2053-3249',
                  'Cook_Inlet_75000600020024': '247-41-10200-2053-3150',
                  'Cook_Inlet_75000600039012': '247-41-10200-2053-3250',
                  'Cook_Inlet_75000200008668': '247-41-10200-2250-3020',
                  'Cook_Inlet_75000200010216': '247-41-10200-2015-3035-4223',
                  'Cook_Inlet_75000200010370': '247-41-10200',
                  'Cook_Inlet_75000200018431': '247-41-10100-2005',
                  'Cook_Inlet_75005300016711': '245-50-10120',
                  'Cook_Inlet_75005300002575': '245-50-10010',
                  'Cook_Inlet_75005300019947': '247-10-10080-2010-3068',
                  'Cook_Inlet_75005300002502': '247-10-10080-2010',
                  'Cook_Inlet_75005300009788': '247-10-10080-2020-3029',
                  'Cook_Inlet_75005300010442': '247-10-10080',
                  'Cook_Inlet_75005300005137': '245-50-10050-2016-3044',
                  'Cook_Inlet_75005300029149': '245-50-10020',
                  'Cook_Inlet_75005300003369': '247-30-10090',
                  'Cook_Inlet_75005300027433': '247-10-10080-2010-3060',
                  'Cook_Inlet_75005400035306': '245-30-10010',
                  'Cook_Inlet_75005400033392': '248-20-10070',
                  'Cook_Inlet_75005400040621': '245-30-10090',
                  'Cook_Inlet_75005400017221': '245-30-10120',
                  'Cook_Inlet_75005400045707': '248-30-10010',
                  'Cook_Inlet_75005400010388': '248-30-10020',
                  'Cook_Inlet_75005400012302': '243-10-10030',
                  'Cook_Inlet_75005400032742': '248-40-10100-2040',
                  'Cook_Inlet_75005400039143': '248-40-10100',
                  'Cook_Inlet_75005400038974': '248-20-10040',
                  'Cook_Inlet_75005400031128': '243-10-10050',
                  'Cook_Inlet_75005400029404': '245-40-10050',
                  'Copper_River_75019800015373': '212-20-10080',
                  'Copper_River_75019800001873': '212-20-10080',
                  'Copper_River_75019800007832': '212-20-10080',
                  'Copper_River_75019600181061': '212-20-10080-2461-3292',
                  'Copper_River_75019600161870': '212-20-10080-2404',
                  'Copper_River_75019600128599': '212-20-10080',
                  'Copper_River_75019600197497': '212-20-10080-2331-3093',
                  'Copper_River_75019700016379': '212-20-10080-2300-3660-4258',
                  'Copper_River_75019700016412': '212-20-10080-2300-3660-4258-5050',
                  'Copper_River_75019700010633': '212-20-10080-2300-3511-4035',
                  'Copper_River_75019700010617': '212-20-10080-2300',
                  'Copper_River_75019700007155': '212-20-10080-2300-3511-4110',
                  'Copper_River_75019700006876': '212-20-10080-2300',
                  'Copper_River_75019700012433': '212-20-10080-2300-3310',
                  'Copper_River_75019700010666': '212-20-10080-2300-3511',
                  'Copper_River_75003900065861': '212-20-10080-2055',
                  'Copper_River_75003900049180': '212-20-10040-2011',
                  'Copper_River_75003900055856': '212-20-10040',
                  'Copper_River_75003900084262': '212-20-10080',
                  'Prince_William_Sound_18357': '227-10-17110',
                  'Prince_William_Sound_30774': '225-20-15030',
                  'Prince_William_Sound_41123': '221-30-10560',
                  'Prince_William_Sound_41143': '221-30-10655',
                  'Prince_William_Sound_43513': '221-30-10517',
                  'Prince_William_Sound_43793': '221-30-10480',
                  'Prince_William_Sound_44113': '221-20-10350',
                  'Prince_William_Sound_45863': '221-20-10260',
                  'Prince_William_Sound_46065': '228-50-18190',
                  'Prince_William_Sound_62232': '222-20-12242',
                  'Prince_William_Sound_66912': '221-50-11060',
                  'Prince_William_Sound_72972': '221-40-10990',
                  'Prince_William_Sound_82471': '221-60-11380-2095',
                  'Copper_River_75003900034359': '212-10-10020',
                  'Cook_Inlet_75000600015299':'247-41-10200-2053-3150',
                  'Cook_Inlet_75005300022592':'247-30-10090-2151',
                  'Cook_Inlet_75005400039351':'248-40-10105',
                  'Copper_River_75003900084672':'212-20-10080-2100-3089',
                  'Copper_River_75003900054448':'228-70-10500',
                  'Prince_William_Sound_39255':'224-40-14904',
                  'Prince_William_Sound_60732':'223-30-13180',
                  'Prince_William_Sound_78912':'222-10-12040'
                  }

# Cats that were missed but should be associated with an awc lake record
awcHuc12LakesQcLakesDict = {
                           }

# Dictionary to relate catchments to awc using awc OIDs for those missed in spatial join, watershed is correct but awc line did not intersect outlet catchment.
addHuc12QcDict = {'Cook_Inlet_75004200003619':['20661','20746','21215','22051'],
                  'Prince_William_Sound_78912':['29546','30080','30343','30608'],
                  'Prince_William_Sound_39255':['28534','28669'],
                  'Prince_William_Sound_60732':['28070'],
                  'Copper_River_75003900054448':['29073','29201','29280','30050','30133'],
                  'Copper_River_75003900084672':['30582'],
                  'Cook_Inlet_75005300022592':['26431'],
                  'Cook_Inlet_75005400039351':['12237','12953'],
                  'Cook_Inlet_75000600015299':['30863','31442','31536','31760','32110','32597'],
                  'Cook_Inlet_75004200016346':['21668'],
                  'Cook_Inlet_75004200012680':['22252','22404'],
                  'Cook_Inlet_75004200005429':['20580'],
                  'Cook_Inlet_75004200012771':['20608','20671','20894','21542','21682','22151'],
                  'Cook_Inlet_75004200006975':['20808','20946','21328','21462','21569','21662','21736','22023','22198'],
                  'Cook_Inlet_75004300006970':['20471','20625','20711','20913','20938','21141','21608','22191','22391'],
                  'Cook_Inlet_75004300002057':['21250','21295','22039'],
                  'Cook_Inlet_75004300007978':['18653','19249','19635','20198'],
                  'Cook_Inlet_75004300001135':['18194','19433','19603','19984','20340'],
                  'Cook_Inlet_75004300006302':['20654','20739','20811','20878','22135'],
                  'Cook_Inlet_75004300002183':['12185','12958'],
                  'Cook_Inlet_75004300005952':['20482','21102','21152','22394'],
                  'Cook_Inlet_75004400007224':['27445','27587','27931','28237','28656'],
                  'Cook_Inlet_75004400000334':['24422','24714','24964','25460','25481','25563','25756'],
                  'Cook_Inlet_75004400000370':['27720','28378'],
                  'Cook_Inlet_75004400003230':['20542','21948','22172'],
                  'Cook_Inlet_75004400001670':['28363'],
                  'Cook_Inlet_75000100004048':['33457','33912','34455','34669','34858','34962','35155'],
                  'Cook_Inlet_75000100002482':['26579'],
                  'Cook_Inlet_75000100004383':['32996','33227','33575','33632','34172','34973'],
                  'Cook_Inlet_75000400014408':['33736','33902','34679','34740','35165'],
                  'Cook_Inlet_75000400000017':['33068','33279','33659','34330','34494'],
                  'Cook_Inlet_75000400000163':['33316','33882','34767'],
                  'Cook_Inlet_75000700031304':['36420'],
                  'Cook_Inlet_75000700046720':['27009','27197','27668','27827','28158','28225'],
                  'Cook_Inlet_75000500006217':['35350','36473','36606','36695','37240'],
                  'Cook_Inlet_75000500002477':['35350','36282','36473','36606','36695','37240'],
                  'Cook_Inlet_75000600029695':['30958','31901'],
                  'Cook_Inlet_75000600027457':['30722','31822','32118','32763','32952'],
                  'Cook_Inlet_75000600002850':['32435'],
                  'Cook_Inlet_75000600020073':['31065','31964','32006','32456','32809'],
                  'Cook_Inlet_75000600027807':['31855'],
                  'Cook_Inlet_75000600020024':['32022','32585'],
                  'Cook_Inlet_75000600039012':['31641','31747'],
                  'Cook_Inlet_75000200008668':['33219','33894','34732','34840'],
                  'Cook_Inlet_75000200010216':['31946','32775'],
                  'Cook_Inlet_75000200010370':['27009','27197','27434','2750','27668','27827','28225','28556','28645'],
                  'Cook_Inlet_75000200018431':['26885','28430'],
                  'Cook_Inlet_75005300016711':['24551','25423','26283'],
                  'Cook_Inlet_75005300002575':['24545','24571','25723','26187','26741'],
                  'Cook_Inlet_75005300019947':['24574','25394','25727'],
                  'Cook_Inlet_75005300002502':['24443','24737','24907','25091','25124','25203','25586','26055','26457'],
                  'Cook_Inlet_75005300009788':['24468','24874','25895','26002','26081','26359','26573'],
                  'Cook_Inlet_75005300010442':['24509','24892','24971','24998','25186','25682','25762','25854','26039'],
                  'Cook_Inlet_75005300005137':['25985'],
                  'Cook_Inlet_75005300029149':['25438','26091'],
                  'Cook_Inlet_75005300003369':['24526','24818','25338','25781','25950','25981','26388'],
                  'Cook_Inlet_75005300027433':['24991','25239','25347'],
                  'Cook_Inlet_75005400035306':['18227','18608','18707','19062','20120','20232'],
                  'Cook_Inlet_75005400033392':['19023','19245','19323'],
                  'Cook_Inlet_75005400040621':['18898'],
                  'Cook_Inlet_75005400017221':['18334'],
                  'Cook_Inlet_75005400045707':['18169','18628','19005'],
                  'Cook_Inlet_75005400010388':['18275','18736','19763'],
                  'Cook_Inlet_75005400012302':['11529','12346','12428','12458','12887','13146'],
                  'Cook_Inlet_75005400032742':['11766','12060'],
                  'Cook_Inlet_75005400039143':['11859','12352'],
                  'Cook_Inlet_75005400038974':['19301','19388','20130','20339'],
                  'Cook_Inlet_75005400031128':['10743','11237','11559','11938'],
                  'Cook_Inlet_75005400029404':['18814','19385','19631','19664','20090'],
                  'Copper_River_75019800015373':['29614','29821'],
                  'Copper_River_75019800001873':['29821'],
                  'Copper_River_75019800007832':['29614','29821'],
                  'Copper_River_75019600181061':['36233'],
                  'Copper_River_75019600161870':['33024'],
                  'Copper_River_75019600128599':['28901','29408','29475','29614','29821','30538'],
                  'Copper_River_75019600197497':['34150'],
                  'Copper_River_75019700016379':['29090'],
                  'Copper_River_75019700016412':['29009'],
                  'Copper_River_75019700010633':['33929'],
                  'Copper_River_75019700010617':['33071','33176','33662','34573'],
                  'Copper_River_75019700007155':['33383','34186'],
                  'Copper_River_75019700006876':['33071','34573'],
                  'Copper_River_75019700012433':['33209','33534','33883','34829','35036'],
                  'Copper_River_75019700010666':['34765'],
                  'Copper_River_75003900065861':['29159','29720','30152'],
                  'Copper_River_75003900049180':['29412','29492','29750','29839','30409','30599'],
                  'Copper_River_75003900055856':['28905','29253','29951','30334'],
                  'Copper_River_75003900084262':['28901','29105','29269','29408','29475','29614','29821','30165','30538'],
                  'Prince_William_Sound_18357':['20697','21594','21911','22154'],
                  'Prince_William_Sound_30774':['27499'],
                  'Prince_William_Sound_41123':['26856','27269'],
                  'Prince_William_Sound_41143':['28419'],
                  'Prince_William_Sound_43513':['28851','30379'],
                  'Prince_William_Sound_43793':['29114','29258','29829'],
                  'Prince_William_Sound_44113':['29560','29832'],
                  'Prince_William_Sound_45863':['28956','30138'],
                  'Prince_William_Sound_46065':['26914','27003','27094','27171','27399','27731'],
                  'Prince_William_Sound_62232':['28495','28728'],
                  'Prince_William_Sound_66912':['27524','28198'],
                  'Prince_William_Sound_72972':['27343','28560'],
                  'Prince_William_Sound_82471':['27738','28161'],
                  'Copper_River_75003900034359':['29613','30251'],
                  }

# Dictionary to relate huc 12 outlets without awc data to correct catchment/awc - new watersheds/covariates need to be calculated
wrongWtd = {'Cook_Inlet_75004200001726':['Cook_Inlet_75004200002199','231-30-10080'],
            'Cook_Inlet_75004200004105':['Cook_Inlet_75004200000559','226-50-16226'],
            'Cook_Inlet_75004200001298':['Cook_Inlet_75004200010496','232-40-10190'],
            'Cook_Inlet_75004200006640':['Cook_Inlet_75004200010672','242-41-10380'],
            'Cook_Inlet_75004200009218':['Cook_Inlet_75004200000848','232-30-10247'],
            'Cook_Inlet_75004200013577':['Cook_Inlet_75004200012465','232-23-10100'],
            'Cook_Inlet_75004200000478':['Cook_Inlet_75004200005907','231-30-10160'],
            'Cook_Inlet_75004200001167':['Cook_Inlet_75004200003332','232-10-10330'],
            'Cook_Inlet_75004300007319':['Cook_Inlet_75004300003397','241-12-10100'],
            'Cook_Inlet_75000400029059':['Cook_Inlet_75000400033841','247-50-10200-2081-3025'],
            'Cook_Inlet_75000500006404':['Cook_Inlet_75000500002272','247-41-10200-2381-3161-4080-5001'],
            'Cook_Inlet_75000300026273':['Cook_Inlet_75000300022808','247-41-10200-2370-3297'],
            'Cook_Inlet_75000300000174':['Cook_Inlet_75000300021956','247-41-10200-2370'],
            'Cook_Inlet_75000600015375':['Cook_Inlet_75000600040122','247-41-10200-2053-3220'],
            'Cook_Inlet_75000600018321':['Cook_Inlet_75000600018460','247-41-10200-2053-3205-4099'],
            'Cook_Inlet_75000600026866':['Cook_Inlet_75000600006507','247-41-10200-2053-3229-4200'],
            'Cook_Inlet_75000600009692':['Cook_Inlet_75000600025058','247-41-10200-2053'],
            'Cook_Inlet_75000200008552':['Cook_Inlet_75000200005953','247-41-10200-2130-3020'],
            'Cook_Inlet_75000200002996':['Cook_Inlet_75000200018317','247-41-10200-2081-3100-4136'],
            'Cook_Inlet_75000200012956':['Cook_Inlet_75000200000235','247-41-10200'],
            'Cook_Inlet_75000200003487':['Cook_Inlet_75000200018781','247-41-10200'],
            'Cook_Inlet_75000200003483':['Cook_Inlet_75000200008607','247-41-10200-2250'],
            'Cook_Inlet_75005300003320':['Cook_Inlet_75005300007002','247-30-10090-2150'],
            'Cook_Inlet_75005300018781':['Cook_Inlet_75005300022592','247-30-10090-2151'],
            'Cook_Inlet_75005400022563':['Cook_Inlet_75005400034611','245-30-10084'],
            'Cook_Inlet_75005400019698':['Cook_Inlet_75005400008084','243-50-10020'],
            'Cook_Inlet_75005400000004':['Cook_Inlet_75005400029492','245-20-10230'],
            'Copper_River_75019600174661':['Copper_River_75019600133175','212-20-10080-2401'],
            'Copper_River_75019600198622':['Copper_River_75019600133538','212-20-10080-2331'],
            'Copper_River_75003900024361':['Copper_River_75003900057807','212-20-10080-2100-3113'],
            'Copper_River_75003900070258':['Copper_River_75003900034185','212-20-10080-2159-3021'],
            'Copper_River_75003900008591':['Copper_River_75003900084672','212-20-10080-2100-3089'],
            'Copper_River_75003900051393':['Copper_River_75003900084267','212-20-10080-2021'],
            'Copper_River_75003900055093':['Copper_River_75003900023613','228-20-18420'],
            'Copper_River_75003900054944':['Copper_River_75003900054448','228-70-10500'],
            'Prince_William_Sound_18747':['Prince_William_Sound_18967','227-30-17780'],
            'Prince_William_Sound_27856':['Prince_William_Sound_19416','226-40-16851'],
            'Prince_William_Sound_30084':['Prince_William_Sound_29704','224-10-14540'],
            'Prince_William_Sound_33106':['Prince_William_Sound_16467','226-50-16154'],
            'Prince_William_Sound_42005':['Prince_William_Sound_39255','224-40-14904'],
            'Prince_William_Sound_42595':['Prince_William_Sound_41025','224-40-14900'],
            'Prince_William_Sound_43085':['Prince_William_Sound_40635','225-30-15070'],
            'Prince_William_Sound_43255':['Prince_William_Sound_37935','226-20-16010'],
            'Prince_William_Sound_44273':['Prince_William_Sound_35343','223-40-13390'],
            'Prince_William_Sound_44605':['Prince_William_Sound_37955','226-20-16950'],
            'Prince_William_Sound_45925':['Prince_William_Sound_28734','224-40-14860'],
            'Prince_William_Sound_46613':['Prince_William_Sound_36553','224-10-14510'],
            'Prince_William_Sound_46623':['Prince_William_Sound_34993','224-10-14500'],
            'Prince_William_Sound_70762':['Prince_William_Sound_61812','222-10-12030'],
            'Prince_William_Sound_75002':['Prince_William_Sound_60732','223-30-13180'],
            'Prince_William_Sound_77992':['Prince_William_Sound_75842','224-10-14020'],
            'Prince_William_Sound_80112':['Prince_William_Sound_78912','222-10-12040'],
            'Prince_William_Sound_84641':['Prince_William_Sound_72242','222-20-12640'],
            'Prince_William_Sound_89681':['Prince_William_Sound_75151','221-60-11520'],
            }

print(f'QC Dictionaries set\n{"*"*100}')

QC Dictionaries set
****************************************************************************************************


In [53]:
# Relate awc code back to new catchment using one of the awc oids stored in the dictionary
# Copy this and add back to awcQC dictionary or modify update sp_ls code block below
for k,v in addHuc12QcDict.items():
    #print(k,v)
    awccode = awcOidDict[int(v[0])][1]
    print (f"'{k}':'{awccode}',")

'Cook_Inlet_75004200003619':'232-22-10130',
'Cook_Inlet_75004200016346':'231-20-10337',
'Cook_Inlet_75004200012680':'233-30-10070',
'Cook_Inlet_75004200005429':'231-30-10080-2119',
'Cook_Inlet_75004200012771':'242-31-10120',
'Cook_Inlet_75004200006975':'226-50-16286',
'Cook_Inlet_75004300006970':'241-14-10645',
'Cook_Inlet_75004300002057':'241-14-10645-2131',
'Cook_Inlet_75004300007978':'241-14-10600',
'Cook_Inlet_75004300001135':'241-30-10500',
'Cook_Inlet_75004300006302':'241-20-10550',
'Cook_Inlet_75004300002183':'241-40-10300',
'Cook_Inlet_75004300005952':'241-14-10660',
'Cook_Inlet_75004400007224':'247-60-10210',
'Cook_Inlet_75004400000334':'244-30-10010',
'Cook_Inlet_75004400000370':'244-30-10010-2140',
'Cook_Inlet_75004400003230':'244-30-10010-2076-3095',
'Cook_Inlet_75004400001670':'244-30-10010-2096',
'Cook_Inlet_75000100004048':'247-50-10175',
'Cook_Inlet_75000100002482':'247-41-10080',
'Cook_Inlet_75000100004383':'247-50-10330',
'Cook_Inlet_75000400014408':'247-50-10200',
'C

In [82]:
# #'Cook_Inlet_75004200012620': ['Cook_Inlet_75004200001574', '232-21-10240', '190202021202', 2],
# reviewDict2={}
# with arcpy.da.SearchCursor(awcjoin,['cat_ID_con', 'HUC12'])as cur:
#     for row in cur:
#         if row[0] in wrongWtd:
#             newcat = wrongWtd[row[0]][0]
#             awccode = wrongWtd[row[0]][1]
#             #print(f'HUC {row[1]} catchment {row[0]} is incorrect and will be changed to {newcat} and awcEvent {awccode}')
#             append_value(reviewDict2,row[0],[newcat,awccode,row[1],2])
# for k, v in reviewDict2.items():
#     print(f"'{k}':{v},")


'Cook_Inlet_75004200001726':['Cook_Inlet_75004200002199', '231-30-10080', '190202020508', 2],
'Cook_Inlet_75004200001298':['Cook_Inlet_75004200010496', '232-40-10190', '190202020705', 2],
'Cook_Inlet_75004200006640':['Cook_Inlet_75004200010672', '242-41-10380', '190202021303', 2],
'Cook_Inlet_75004200009218':['Cook_Inlet_75004200000848', '232-30-10247', '190202020802', 2],
'Cook_Inlet_75004200013577':['Cook_Inlet_75004200012465', '232-23-10100', '190202020902', 2],
'Cook_Inlet_75004200000478':['Cook_Inlet_75004200005907', '231-30-10160', '190202020510', 2],
'Cook_Inlet_75004200001167':['Cook_Inlet_75004200003332', '232-10-10330', '190202021103', 2],
'Cook_Inlet_75004300007319':['Cook_Inlet_75004300003397', '241-12-10100', '190203010807', 2],
'Cook_Inlet_75004300001512':['Cook_Inlet_75004300005173', '244-20-10090', '190203010801', 2],
'Cook_Inlet_75000400029059':['Cook_Inlet_75000400033841', '247-50-10200-2081-3025', '190204021206', 2],
'Cook_Inlet_75000500006404':['Cook_Inlet_750005000

## Update using qc dictionaries after reviewing new watersheds
Use AWC_OID codes to update

In [60]:
# List to store catchments that were did not get attributed in spatial join but were identified in initial review as awc catchments
adQC = []
# List to store catchments without spawning habitat
nosphab = []
sp_ls_missvals = []
sa_sp_codes = ['CO_s','S_s','K_s','CH_s','P_s']
sa_no_sp_codes = ['CO_p','CO_r','S_p','S_r','K_p','K_r','CH_p','CH_r','P_r','P_p']
fields = ['cat_ID_con','AWC_CODE_qc','sp_ls','sa_sp_hab','AWC_OID','AWC_CODE','mult_awc','LSTAGE','SPECIES','sa_sp_count']

c=1
with arcpy.da.UpdateCursor(awcjoin,fields) as cur:
    for row in cur:
        if row[0] in awcHuc12QcDict:
            print(f'\n{("*"*100)}')
            print(f'{c}. {row[0]} found in awc qc dictionary')
            updCode = awcHuc12QcDict[row[0]]
            if updCode is not None:
                if row[4] is  not None:
                    awcOidList = []
                    awcSpLS = []
                    awcOIDvals = row[4].split(",")
                    print(f' oid vals  = {awcOIDvals} - row4 = {row[4]}')
                    awcCodevals = row[5].split(",")
                    print(f'code vals = {awcCodevals}')
                    snum = len(row[5].split(","))
                    for i in range(snum):
                        if awcCodevals[i] == updCode:
                            awcoid = int(awcOIDvals[i])
                            print(f'awcCode value of i = {awcCodevals[i]} and updCode = {updCode}\nAWC_OID = {awcoid}')
                            print(f'Appending {awcOidDict[int(awcOIDvals[i])][0]} from OID dictionary')
                            awcSpLS.append(awcOidDict[int(awcOIDvals[i])][0])
                    updSpLS = str(awcSpLS).strip("[]").replace("'","")
                    print (f'Awc codes {row[1]} will be updated to {updCode}')
                    print (f'Sp_ls codes {row[2]} will be updated to {updSpLS}')
                    if len(set(awcSpLS).intersection(sa_sp_codes)) >0:
                        print(f'{row[0]} has {len(set(awcSpLS).intersection(sa_sp_codes))} salmon spawning records')
                        saspcount = len(set(awcSpLS).intersection(sa_sp_codes))
                        row[3] = 1
                        row[9] = saspcount
                    elif len(set(awcSpLS).intersection(sa_no_sp_codes)) > 0  and len(set(awcSpLS).intersection(sa_sp_codes)) == 0:
                        print(f'{row[0]} has {len(set(awcSpLS).intersection(sa_no_sp_codes))} salmon species records')
                        row[3] = 2
                        row[9] = 0
                    else:
                        print(f'{row[0]} has no spawning habitat recorded')
                        nosphab.append(row[0])
                        row[3] = 0
                        row[9] = 0
                    row[1] = updCode
                    row[2] = updSpLS
                else:
                    sp_ls_missvals = []
                    print(f'{row[0]} missed during spatial join but identified during qc - must review again and add OBJECTIDs from awc to new keyval dictionary')
                    for val in addHuc12QcDict[row[0]]:
                        intval = int(val)
                        sp_ls_val = awcOidDict[intval][0]
                        sp_ls_missvals.append(sp_ls_val)
                    row[1] = updCode
                    # sp_ls
                    sp_ls_up = str(sp_ls_missvals).strip("[]").replace("\'","")
                    print(f'{sp_ls_up}')
                    row[2] = sp_ls_up
                    # sa_sp_hab
                    if len(set(sp_ls_missvals).intersection(sa_sp_codes)) >0:
                        print(f'{row[0]} has {len(set(sp_ls_missvals).intersection(sa_sp_codes))} salmon spawning records')
                        saspcount = len(set(sp_ls_missvals).intersection(sa_sp_codes))
                        row[3] = 1
                        row[9] = saspcount
                    elif len(set(sp_ls_missvals).intersection(sa_no_sp_codes)) > 0  and len(set(sp_ls_missvals).intersection(sa_sp_codes)) == 0:
                        print(f'{row[0]} has {len(set(sp_ls_missvals).intersection(sa_no_sp_codes))} salmon species records')
                        row[3] = 2
                        row[9] = 0
                    else:
                        print(f'{row[0]} has no spawning habitat recorded')
                        nosphab.append(row[0])
                        row[3] = 0
                        row[9] = 0
                    # AWC_OID
                    oids = str(addHuc12QcDict[row[0]]).strip("[]").replace("'","")
                    print(oids)
                    row[4] = oids
                    # AWC_CODE
                    print(len(addHuc12QcDict))
                    awccode = str(awcHuc12QcDict[row[0]].replace("'",""))

                    print(awccode)
                    row[5] = awccode
                    adQC.append(row[0])
            else:
                print(f'{("*"*100)}\n')
                print(f'{row[0]} will have code removed {updCode}')
                row[1]=None
                row[2]=None
                row[3]=None
                row[4]=None
                row[5]=None
                row[6]=None
                row[7]=None
                row[8]=None
                row[9]=None

            c+=1
        elif row[0] in awcHuc12LakesQcLakesDict:
            print(f'{("*"*100)}\n')
            print(f'{c}. {row[0]} found in awc lakes qc dictionary')
            print(f'{row[0]} found in awc qc dictionary')
            updCode = awcHuc12LakesQcLakesDict[row[0]]
            updSpLS = str(awcLakesDict[updCode]).strip("[]").replace("'","")
            print (f'Awc codes {row[1]} will be updated to {updCode}')
            print (f'Sp_ls codes {row[2]} will be updated to {updSpLS}')
            if len(set(awcLakesDict[updCode]).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {len(set(awcLakesDict[updCode]).intersection(sa_sp_codes))} salmon spawning records')
                saspcount = len(set(awcLakesDict[updCode]).intersection(sa_sp_codes))
                row[3] = 1
                row[9] = saspcount
            elif len(set(awcLakesDict[updCode]).intersection(sa_no_sp_codes)) > 0  and len(set(awcLakesDict[updCode]).intersection(sa_sp_codes)) == 0:
                print(f'{row[0]} has {len(set(awcLakesDict[updCode]).intersection(sa_no_sp_codes))} salmon species records')
                row[3] = 2
                row[9] = 0
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[3] = 0
                row[9] = 0
            row[1] = updCode
            row[2] = updSpLS
            c+=1
        else:
            pass
        cur.updateRow(row)
    del(row)
del(cur)


****************************************************************************************************
1. Cook_Inlet_75004200001493 found in awc qc dictionary
 oid vals  = ['20465', '20933', '21315', '22101'] - row4 = 20465,20933,21315,22101
code vals = ['233-20-10080', '233-20-10082', '233-20-10082', '233-20-10080']
awcCode value of i = 233-20-10080 and updCode = 233-20-10080
AWC_OID = 20465
Appending CH_s from OID dictionary
awcCode value of i = 233-20-10080 and updCode = 233-20-10080
AWC_OID = 22101
Appending P_s from OID dictionary
Awc codes 233-20-10080 will be updated to 233-20-10080
Sp_ls codes CH_s, P_s will be updated to CH_s, P_s
Cook_Inlet_75004200001493 has 2 salmon spawning records

****************************************************************************************************
2. Cook_Inlet_75004200003619 found in awc qc dictionary
 oid vals  = ['20661', ' 20746', ' 21215', ' 22051'] - row4 = 20661, 20746, 21215, 22051
code vals = ['232-22-10130']
awcCode value of i = 

## Join fields to tempsites using cat_ID_con
* Join Sp_Ls and sa_sp_hab to point fc of temperature sites

In [61]:
#sites = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_all_sites_sj_merge"
#sites = r"D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\all_sites_sj_merge"

print(sites)
sitefields = [f.name for f in arcpy.ListFields(sites)]
sitefields

D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_outlet_cats_points


['OBJECTID',
 'Shape',
 'cat_ID_con',
 'dist_catch_coast_km',
 'HUC12',
 'HUC12_Review',
 'DsContAreaSqKm',
 'MERGE_SRC']

In [62]:
from time import strftime
print( "Start script: " + strftime("%Y-%m-%d %H:%M:%S"))

sitefields = [f.name for f in arcpy.ListFields(sites)]
awctextfields = ['sp_ls','AWC_CODE']
awcshortfields = ['sa_sp_hab','sa_sp_count']
sourceFieldsList = ['cat_ID_con','sp_ls','sa_sp_hab','sa_sp_count','AWC_CODE_qc']
updateFieldsList = ['cat_ID_con','sp_ls','sa_sp_hab','sa_sp_count','AWC_CODE']
# Use list comprehension to build a dictionary from a da SearchCursor
valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(awcjoin, sourceFieldsList)}

awcCatsDict = {}
for awctextfield in awctextfields:
    if awctextfield not in sitefields:
        print(f'adding field {awctextfield} to {sites}')
        arcpy.AddField_management(sites,awctextfield,'TEXT','','',500)
    else:
        print(f'{awctextfield} present')

for awcshortfield in awcshortfields:
    if awcshortfield not in sitefields:
        print(f'adding field {awcshortfield} to {sites}')
        arcpy.AddField_management(sites,awcshortfield,'SHORT')
    else:
        print(f'{awcshortfield} present')

with arcpy.da.UpdateCursor(sites,updateFieldsList) as cur:
    for row in cur:
        keyVal = row[0]
        if keyVal in valueDict:
            for n in range (1,len(sourceFieldsList)):
                row[n] = valueDict[keyVal][n-1]
            cur.updateRow(row)

del valueDict

print( "Finished script: " + strftime("%Y-%m-%d %H:%M:%S"))


Start script: 2022-04-15 11:40:51
adding field sp_ls to D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_outlet_cats_points
adding field AWC_CODE to D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_outlet_cats_points
adding field sa_sp_hab to D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_outlet_cats_points
adding field sa_sp_count to D:\\GIS\\AKSSF_awcHuc12_cv_Final\\AKSSF_awcHuc12_cv_Final.gdb\\AKSSF_awcHuc12_outlet_cats_points
Finished script: 2022-04-15 11:40:53


# Updated to add to code above for catchments as well as sites.
No need to run unless re-using to update another dataset
~~## Update sp_ls_hab code as follows~~
* 1 for salmon spawning
* 2 for salmon species but no spawning
* 0 for awc records but neither salmon spawning nor any other combo of salmon sp_ls
* NULL no awc data at all~~

In [63]:
# sa_no_sp_codes = ['CO_p','CO_r','S_p','S_r','K_p','K_r','CH_p','CH_r','P_r','P_p']
# sa_sp_codes = ['CO_s','S_s','K_s','CH_s','P_s']
# #sites = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_all_sites_sj_merge"
# sites = r"D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\all_sites_sj_merge"
# noSalmList = []
#
# with arcpy.da.UpdateCursor(sites, ['sp_ls','cat_ID_con','SiteID','sa_sp_hab']) as cur:
#     for row in cur:
#         if row[0] is not None:
#             print(f'{("*")*100}')
#             print(f'Site {row[2]} has {row[0]}')
#             sp_lsLs = row[0].replace(" ","").split(",")
#             print(sp_lsLs)
#             if len(set(sp_lsLs).intersection(sa_sp_codes)) > 0:
#                 print(f'{row[0]} has {len(set(sp_lsLs).intersection(sa_sp_codes))} salmon spawning records')
#                 row[3] = 1
#             elif len(set(sp_lsLs).intersection(sa_no_sp_codes)) > 0  and len(set(sp_lsLs).intersection(sa_sp_codes)) == 0:
#                 print(f'{row[0]} has {len(set(sp_lsLs).intersection(sa_no_sp_codes))} salmon species records')
#                 row[3] = 2
#             else:
#                 print(f'{row[0]} has no salmon spawning habitat or species presence #####################')
#                 noSalmList.append(row[1])
#                 row[3] = 0
#         cur.updateRow(row)
#     del(row)
# del(cur)
#
# print('complete')
# noSalmList
